# Digit Recognizer: Vision Fundamentals

_Version 1: 05/01/18_

This is an analysis of the MNIST digit data set using a Convolution Neural Network (CNN).  The CNN is implemented with Keras on top of Tensorflow.

The data set can be found on [kaggle](https://www.kaggle.com/c/digit-recognizer).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Convolution2D
from keras.layers import MaxPooling2D, Flatten
from keras.utils import np_utils

np.random.seed(149)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

/Users/craigkleski/Library/Python/3.6/lib/python/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Read and format the data for use in the model.

In [2]:
X = train[~train['label'].isnull()].drop(columns=['label'])
y = train['label']

# Use only underlying numpy array
X = X.values

# each image is 28x28. to use CNN, the data must be reshaped
X = X.reshape(-1,28, 28, 1)

# rescale the pixel values to be in the interval [0,1]
X = X.astype(float)
X /= 255

# create train-test split. X_te and y_te are our validation set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

# one-hot encode the 10 digit categories
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

Let's check the shape of X_tr and y_tr to make sure it looks OK.

In [3]:
print(X_train.shape, y_train.shape)

(37800, 28, 28, 1) (37800,)


Now we create the CNN. The Keras `Sequential` model stacks linear layers. 

In [4]:
model = Sequential()

# for the initial convolution we specify the input shape
model.add(Convolution2D(32, (5, 5), activation='relu', input_shape=(28,28,1)))

# add convolution and pooling layers. dropout to prevent overfitting
model.add(Convolution2D(32, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flattens the input array
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# final layer for 10 prediction classes
model.add(Dense(10, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
# fit model on training set
model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

# see how we did
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

Epoch 1/10
37800/37800 [==============================] - 163s 4ms/step - loss: 0.2366 - acc: 0.9251
Epoch 2/10
37800/37800 [==============================] - 159s 4ms/step - loss: 0.0862 - acc: 0.9733
Epoch 3/10
37800/37800 [==============================] - 160s 4ms/step - loss: 0.0649 - acc: 0.9798
Epoch 4/10
37800/37800 [==============================] - 166s 4ms/step - loss: 0.0573 - acc: 0.9830
Epoch 5/10
37800/37800 [==============================] - 178s 5ms/step - loss: 0.0483 - acc: 0.9854
Epoch 6/10
37800/37800 [==============================] - 184s 5ms/step - loss: 0.0453 - acc: 0.9868
Epoch 7/10
37800/37800 [==============================] - 187s 5ms/step - loss: 0.0415 - acc: 0.9866
Epoch 8/10
37800/37800 [==============================] - 189s 5ms/step - loss: 0.0359 - acc: 0.9894
Epoch 9/10
37800/37800 [==============================] - 191s 5ms/step - loss: 0.0346 - acc: 0.9894
Epoch 10/10
37800/37800 [==============================] - 192s 5ms/step - loss: 0.0316 - a

I experimented with [BatchNormalization](https://keras.io/layers/normalization/) but it didn't improve my score.

Now reshape the validation set.

In [5]:
X_p = test.values
X_p = X_p.astype(float)
X_p /= 255
X_p = X_p.reshape(-1,28, 28, 1)

In [6]:
pred = model.predict_classes(X_p)
res = pd.DataFrame()
res['ImageId'] = list(test.index+1)
res['Label'] = pred
res.to_csv('digits.csv',index=False)